_Updated date: November 7, 2025_

# 🎓 Databricks for Actuaries: Healthcare Analytics Workshop
**A Hands-On Guide for SAS Users Transitioning to Databricks**

---

## 👋 Welcome Actuaries & Analysts!

This workshop is designed specifically for **actuaries and analysts** who are familiar with **SAS** and want to learn Databricks. Don't worry if you're new to Python or SQL - we'll guide you step by step!

## 📚 Workshop Objectives

By the end of this workshop, you will be able to:

1. ✅ Understand how Databricks compares to your SAS workflows
2. ✅ Load and query healthcare payer data using **simple SQL and PySpark**
3. ✅ Perform actuarial analyses you're familiar with (loss ratios, trends, reserving)
4. ✅ Create interactive visualizations without complex code
5. ✅ Build analytics tables for pricing, reserving, and risk management

---

### 🏥 Dataset Overview

We'll work with **healthcare payer data** including:
- **Members**: Health plan enrollees
- **Claims**: Medical claim submissions
- **Providers**: Healthcare providers
- **Diagnoses**: Diagnosis codes from claims
- **Procedures**: Medical procedures performed

**Think of it as**: Claims = Losses, Members = Policies, Providers = Service Providers

---



## Sample Data Model

For a payer, commonly used tables include:

- **Members**: members enrolled in a health plan
- **Claims**: medical claim submissions
- **Providers**: healthcare providers (doctors, clinics)
- **Diagnoses**: claim diagnosis codes
- **Procedures**: procedures/services performed

Each table should have at least 50 rows.

<img src="https://user-gen-media-assets.s3.amazonaws.com/gpt4o_images/bdd54dc0-f3c7-4975-80a3-0017ebdb121c.png" alt="Managed Tables" width="400" height="300">


# Introduction for Actuaries

## 🤔 Why Databricks for Actuaries?

If you're coming from **SAS**, you might be wondering: "Why learn another tool?"

### Here's Why:
- **Scalability**: Handle millions of claims instantly (no more waiting for PROC SQL!)
- **Modern Analytics**: Built-in ML, real-time dashboards, and collaboration
- **Cost-Effective**: Cloud-based, pay only for what you use
- **Still Use SQL**: 90% of your SAS PROC SQL knowledge transfers directly!

---

## 🔄 SAS vs Databricks: Quick Comparison

| **What You Do in SAS** | **How You Do It in Databricks** | **Difficulty** |
|------------------------|----------------------------------|----------------|
| `PROC SQL` | SQL queries (almost identical!) | ⭐ Easy |
| `PROC MEANS` | `GROUP BY` + aggregate functions | ⭐ Easy |
| `PROC FREQ` | `GROUP BY` + `COUNT()` | ⭐ Easy |
| `DATA` step | SQL `SELECT` or simple Python | ⭐⭐ Moderate |
| `PROC EXPAND` (trending) | Window functions | ⭐⭐ Moderate |
| Macros | Parameters + reusable queries | ⭐⭐⭐ Learning curve |

**Good News**: Most of what you do can be done with **SQL alone**!

---



## 🏠 What is a Lakehouse? (Simple Explanation)

**For Actuaries**: Think of it as a **super-powered SAS library** that:
- Stores all your data in one place (claims, policies, members)
- Lets you analyze it with SQL (like PROC SQL)
- Handles millions of rows instantly
- Keeps track of all changes (audit trail)
- Lets multiple people work at once (no locking issues!)

**Key Benefit**: Unlike SAS datasets, you can query **billions** of claims in seconds!

<img src="https://www.databricks.com/wp-content/uploads/2020/01/data-lakehouse-new.png" alt="Lakehouse" width="500" height="350">

---

## 📚 Unity Catalog (Data Organization - Like SAS Libraries)

**For Actuaries**: Think of Unity Catalog as your **SAS library structure**, but better organized:

```
In SAS:                    In Databricks:
LIBNAME.DATASET            CATALOG.SCHEMA.TABLE
  ↓                           ↓
work.claims        →       my_catalog.payer_bronze.claims
actuarial.loss_ratios →    my_catalog.payer_gold.loss_ratios
```

**Benefits:**
- ✅ Everyone sees the same data (no duplicate datasets!)
- ✅ Built-in security (control who can see PHI/PII)
- ✅ Audit trail (track all data access)
- ✅ Easy to find data (searchable catalog)

<img src="https://www.databricks.com/sites/default/files/2025-05/header-unity-catalog.png?v=1748513086" alt="Unity Catalog" width="500" height="300">

---






## 🥉🥈🥇 Medallion Architecture (Your Data Quality Layers)

**For Actuaries**: This is like your SAS data prep workflow, but organized into layers:

### 📥 Bronze (Raw Data) 
- **Like**: Your raw claims extracts from source systems
- **Contains**: Data exactly as received (CSV, database extracts)
- **Example**: Raw claims file from claims system
- **Today**: We'll load this quickly!

### 🔧 Silver (Cleaned Data)
- **Like**: Your cleaned/standardized SAS datasets
- **Contains**: Deduplicated, standardized data
- **Example**: Claims with proper data types, duplicates removed
- **Today**: We'll auto-clean this!

### ⭐ Gold (Analytics Tables)
- **Like**: Your final analysis datasets (loss triangles, premium summaries)
- **Contains**: Business-ready tables for actuarial analysis
- **Example**: Loss ratios, development factors, IBNR estimates
- **Today**: This is where we'll spend most time! 🎉

<img src="https://www.databricks.com/sites/default/files/inline-images/building-data-pipelines-with-delta-lake-120823.png?v=1702318922" alt="Medallion Architecture" width="500" height="350">


# SETUP
Just run next couple of cells for setup! 

In [0]:
dbutils.widgets.text("catalog", "my_catalog", "Catalog")
dbutils.widgets.text("bronze_db", "payer_bronze", "Bronze DB")
dbutils.widgets.text("silver_db", "payer_silver", "Silver DB")
dbutils.widgets.text("gold_db", "payer_gold", "Gold DB")

catalog = dbutils.widgets.get("catalog")
bronze_db = dbutils.widgets.get("bronze_db")
silver_db = dbutils.widgets.get("silver_db")
gold_db = dbutils.widgets.get("gold_db")

path = f"/Volumes/{catalog}/{bronze_db}/payer/files/"

print(f"Catalog: {catalog}")
print(f"Bronze DB: {bronze_db}")
print(f"Silver DB: {silver_db}")
print(f"Gold DB: {gold_db}")
print(f"Path: {path}")

In [0]:
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")

spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {bronze_db}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {silver_db}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {gold_db}")

spark.sql(f"CREATE VOLUME IF NOT EXISTS {bronze_db}.payer")

# Create the volume and folders
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/claims")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/diagnosis")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/procedures")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/members")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/providers")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/downloads")

In [0]:
import requests
import zipfile
import io
import os
import shutil

# Define the URL of the ZIP file
url = "https://github.com/bigdatavik/databricksfirststeps/blob/6b225621c3c010a2734ab604efd79c15ec6c71b8/data/Payor_Archive.zip?raw=true"

# Download the ZIP file
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Define the base path
base_path = f"/Volumes/{catalog}/{bronze_db}/payer/downloads" 

# Extract the ZIP file to the base path
zip_file.extractall(base_path)

# Define the paths
paths = {
    "claims.csv": f"{base_path}/claims",
    "diagnoses.csv": f"{base_path}/diagnosis",
    "procedures.csv": f"{base_path}/procedures",
    "member.csv": f"{base_path}/members",
    "providers.csv": f"{base_path}/providers"
}

# Create the destination directories if they do not exist
for dest_path in paths.values():
    os.makedirs(dest_path, exist_ok=True)

# Move the files to the respective directories
for file_name, dest_path in paths.items():
    source_file = f"{base_path}/{file_name}"
    if os.path.exists(source_file):
        os.rename(source_file, f"{dest_path}/{file_name}")

# Copy the files to the specified directories and print the paths
shutil.copy(f"{base_path}/claims/claims.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/claims/claims.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/claims/claims.csv")

shutil.copy(f"{base_path}/diagnosis/diagnoses.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/diagnosis/diagnosis.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/diagnosis/diagnosis.csv")

shutil.copy(f"{base_path}/procedures/procedures.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/procedures/procedures.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/procedures/procedures.csv")

shutil.copy(f"{base_path}/members/member.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/members/members.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/members/members.csv")

shutil.copy(f"{base_path}/providers/providers.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/providers/providers.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/providers/providers.csv")


# 🚀 Let's Build Your First Data Pipeline!

---

## Workshop Roadmap

```
📥 Bronze Layer    →    🔧 Silver Layer    →    ⭐ Gold Layer    →    📊 Analytics
   (Raw Data)          (Cleaned Data)        (Business Tables)      (Insights)
```

In the following sections, we'll build a complete data pipeline following the **Medallion Architecture**:

1. **Bronze Layer**: Ingest raw CSV files into Delta tables
2. **Silver Layer**: Clean, deduplicate, and transform data
3. **Gold Layer**: Create enriched analytics tables
4. **Analytics**: Generate insights and visualizations

Let's get started! 🎉

# 📥 Bronze Layer – Ingest Raw Data

---

## What is the Bronze Layer?

The **Bronze Layer** is the landing zone for raw data. Here we:
- 📂 Load data "as-is" from source files (CSV, JSON, Parquet, etc.)
- 💾 Store in Delta Lake format for ACID transactions
- 📝 Apply minimal transformation (just schema inference)
- ⏱️ Keep historical data for audit and reprocessing

> **💡 Best Practice**: Use `COPY INTO` for incremental, idempotent loading from raw data files into Delta Lake tables. It automatically skips already-loaded files!

---



## Step 1: Verify Source Files

Let's first check that our source files are available:

In [0]:
%sql
LIST '/Volumes/my_catalog/payer_bronze/payer/files/claims/'

## Step 2: Load Data with COPY INTO

### 📖 Understanding COPY INTO

`COPY INTO` is Databricks' recommended command for loading data from cloud storage into Delta tables.

**Key Benefits:**
- ✅ **Idempotent**: Safely re-run without duplicating data
- ✅ **Incremental**: Only loads new files automatically
- ✅ **Schema Evolution**: Can merge new columns with `mergeSchema` option
- ✅ **Atomic**: Either succeeds completely or rolls back

**Syntax:**
```sql
COPY INTO <table_name>
FROM '<source_path>'
FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS('mergeSchema' = 'true')
```

📚 **Learn More:**
- [COPY INTO Documentation](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-copy-into)
- [COPY INTO Examples](https://learn.microsoft.com/en-us/azure/databricks/ingestion/cloud-object-storage/copy-into/)


### Loading Data with SQL

In [0]:
%sql
-- Load Claims Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.claims_raw;
COPY INTO payer_bronze.claims_raw FROM
(SELECT
*
FROM '/Volumes/my_catalog/payer_bronze/payer/files/claims/')
FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true', 'force' = 'true');

-- NOTE: 'force = true' is used here for demo purposes only to reload all files every time. In production, omit this option so COPY INTO only processes new data files.


-- Load Diagnosis Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.diagnosis_raw;
COPY INTO payer_bronze.diagnosis_raw FROM
(SELECT
*
FROM '/Volumes/my_catalog/payer_bronze/payer/files/diagnosis/')

FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true');


-- Load Members Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.members_raw;
COPY INTO payer_bronze.members_raw FROM
(SELECT
*
FROM '/Volumes/my_catalog/payer_bronze/payer/files/members/')

FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true');


-- Load Procedures Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.procedures_raw;
COPY INTO payer_bronze.procedures_raw FROM
(SELECT
*
FROM '/Volumes/my_catalog/payer_bronze/payer/files/procedures/')
FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true');


-- Load Providers Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.providers_raw;
COPY INTO payer_bronze.providers_raw FROM
(SELECT
*
FROM '/Volumes/my_catalog/payer_bronze/payer/files/providers/')
FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true');


### 🐍 Alternative: Loading Data with PySpark

While SQL is great for batch loading, PySpark gives you more programmatic control. Here's how to load the same data using PySpark:

In [0]:
# Example: Load data using PySpark
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

# Option 1: Let Spark infer the schema
claims_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/my_catalog/payer_bronze/payer/files/claims/")

# Display first 10 rows
display(claims_df.limit(10))

# Show schema
print("Claims Schema:")
claims_df.printSchema()

# Get row count
print(f"\nTotal rows loaded: {claims_df.count()}")

# Write to Delta table (this creates or replaces the table)
# claims_df.write \
#     .format("delta") \
#     .mode("overwrite") \
#     .saveAsTable("payer_bronze.claims_raw_pyspark")


# 🔧 Silver Layer – Transform, Clean, and Join

---

## What is the Silver Layer?

The **Silver Layer** is where we transform raw data into clean, validated, and enriched datasets. Here we:

- 🧹 **Clean**: Remove nulls, trim whitespace, fix data quality issues
- 🔄 **Transform**: Cast data types, standardize formats
- 🗑️ **Deduplicate**: Remove duplicate records based on business keys
- 🔍 **Validate**: Apply business rules and data quality checks
- 📊 **Enrich**: Join related tables, calculate derived columns

> **💡 Best Practice**: Silver tables should be "analytics-ready" – cleaned, validated, and properly typed!



## Step 1: Transform Bronze to Silver (SQL)

Let's clean and transform our Bronze tables. We'll demonstrate with multiple examples using both **SQL** and **PySpark**.

In [0]:
%sql
-- Create silver schema
CREATE SCHEMA IF NOT EXISTS payer_silver;


-- Members: select relevant fields, cast types, remove duplicates
CREATE OR REPLACE TABLE payer_silver.members AS
SELECT
  DISTINCT CAST(member_id AS STRING) AS member_id,
  TRIM(first_name) AS first_name,
  TRIM(last_name) AS last_name,
  CAST(birth_date AS DATE) AS birth_date,
  gender,
  plan_id,
  CAST(effective_date AS DATE) AS effective_date
FROM payer_bronze.members_raw
WHERE member_id IS NOT NULL;


-- Claims: remove duplicates, prepare data
CREATE OR REPLACE TABLE payer_silver.claims AS
SELECT
  DISTINCT claim_id,
  member_id,
  provider_id,
  CAST(claim_date AS DATE) AS claim_date,
  ROUND(total_charge, 2) AS total_charge,
  LOWER(claim_status) AS claim_status
FROM payer_bronze.claims_raw
WHERE claim_id IS NOT NULL AND total_charge > 0;


-- Providers: deduplicate
CREATE OR REPLACE TABLE payer_silver.providers AS
SELECT
  DISTINCT provider_id,
  npi,
  provider_name,
  specialty,
  address,
  city,
  state
FROM payer_bronze.providers_raw
WHERE provider_id IS NOT NULL;


## Step 2: Transform with PySpark

Now let's see how to do the same transformations using PySpark. This approach is more flexible for complex business logic.

### Example: Transform Procedures Table with PySpark


In [0]:
from pyspark.sql.functions import col, trim, upper, round as spark_round, when, regexp_replace

# Read from Bronze
procedures_bronze = spark.table("payer_bronze.procedures_raw")

# Clean and cast the amount column
procedures_bronze_clean = procedures_bronze.withColumn(
    "amount_clean",
    regexp_replace(col("amount"), "[^0-9.]", "").cast("double")
)

# Apply transformations
procedures_silver = procedures_bronze_clean \
    .dropDuplicates(['claim_id', 'procedure_code']) \
    .filter(col("claim_id").isNotNull()) \
    .filter(col("amount_clean") > 0) \
    .select(
        col("claim_id"),
        upper(trim(col("procedure_code"))).alias("procedure_code"),
        trim(col("procedure_desc")).alias("procedure_desc"),
        spark_round(col("amount_clean"), 2).alias("amount"),
        when(col("amount_clean") < 100, "Low")
        .when(col("amount_clean") < 500, "Medium")
        .when(col("amount_clean") < 1000, "High")
        .otherwise("Very High").alias("cost_category")
    )

# Show sample data
print("Transformed Procedures (first 10 rows):")
display(procedures_silver.limit(10))

# Show statistics
print("\nCost Category Distribution:")
display(procedures_silver.groupBy("cost_category").count().orderBy("cost_category"))

# Write to Silver table
procedures_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("payer_silver.procedures")

# 🤖 Using Databricks AI Assistant

---

Databricks AI Assistant can help you write code, understand data, and troubleshoot issues!

### How to Use AI Assistant:
1. Click the AI Assistant icon
2. Ask questions in natural language
3. Get code suggestions and explanations

### Example Prompts to Try:
- "How do I calculate the total claims by specialty?"
- "Show me how to create a window function for running totals"
- "What does spark.table() command do?"
- "Help me debug this PySpark error"

---



## 🎯 YOUR TURN! (3 mins)
Ask Databricks Assistant: "How do I calculate the total claims by specialty in SQL?"

# ⭐ Gold Layer – Actuarial Analytics (The Fun Part!)

---

## 🎯 What is the Gold Layer? (For Actuaries)

**This is where YOU spend most of your time!** The Gold Layer is like your **final SAS analysis datasets** - ready for actuarial work.

### What We'll Build (Actuarial Examples):

1. **Loss Ratios by Segment**
2. **Claims Trending Analysis**
3. **Claims Development Triangle**
4. **High-Risk Member Identification**
5. **Frequency & Severity Analysis by Demographics**
6. **Data Quality Checks for Actuarial Analysis**
7. **Bias Detection in Healthcare Data**

---

## 🔄 How This Compares to SAS

| **Your SAS Workflow** | **In Databricks Gold Layer** |
|----------------------|------------------------------|
| Create final analysis dataset | Create Gold table |
| PROC SQL with aggregations | SQL SELECT with GROUP BY |
| PROC MEANS for summary stats | Aggregate functions (AVG, SUM, etc.) |
| Multiple DATA steps for calcs | Single SQL statement with CTEs |
| Macros for repeated calcs | Parameterized queries |
| Export to Excel for viz | Built-in interactive charts! |

---

## 🎓 Your Actuarial Toolbox

Today you'll learn SQL equivalents for common actuarial analyses:

- **Loss Ratios**: `SUM(claims)/SUM(premium)`
- **Trending**: Window functions (`LAG`, `LEAD`)
- **Development Factors**: `GROUP BY` claim year + development period
- **Percentiles**: `PERCENTILE_CONT` function
- **Risk Scores**: `CASE WHEN` logic

**Ready?** Let's start building! 🚀

---


## Actuarial Example 1: Loss Ratios by Segment

### 🎯 Business Question
**"What is our loss ratio by provider specialty and state?"**

This is a **fundamental actuarial metric** - you probably calculate this quarterly or annually!

### 📝 SAS vs Databricks

**In SAS, you might write:**
```sas
PROC SQL;
    CREATE TABLE loss_ratios AS
    SELECT 
        p.specialty,
        p.state,
        COUNT(*) AS claim_count,
        SUM(c.total_charge) AS incurred_losses,
        ROUND(SUM(c.total_charge) / COUNT(*), 0.01) AS loss_ratio
    FROM claims AS c
    LEFT JOIN providers AS p
        ON c.provider_id = p.provider_id
    GROUP BY p.specialty, p.state;
QUIT;
```

**In Databricks, we write:**
```sql
-- Very similar! Most SQL transfers directly.
```

Let's build this now! 👇


In [0]:
%sql
-- ACTUARIAL ANALYSIS: Loss Ratios by Specialty and State
CREATE OR REPLACE TABLE payer_gold.loss_ratios AS
SELECT
    p.specialty,
    p.state,
    COUNT(*) AS claim_count,
    SUM(c.total_charge) AS incurred_losses,
    ROUND(SUM(c.total_charge) / COUNT(*), 2) AS loss_ratio
FROM payer_silver.claims c
LEFT JOIN payer_silver.providers p
    ON c.provider_id = p.provider_id
GROUP BY p.specialty, p.state;

-- Display the results including loss_ratio
SELECT 
    specialty,
    state,
    claim_count,
    incurred_losses,
    loss_ratio
FROM payer_gold.loss_ratios;

## 🎯 YOUR TURN! (3 mins)
Using SAS script example below, ask Assistant to convert into Databricks SQL and Pyspark (_Example prompt: Convert below SAS script into Pyspark_)

```sas
PROC SQL;
    CREATE TABLE loss_ratios_summary AS
    SELECT 
        specialty,
        state,
        COUNT(*) AS claim_count,
        SUM(incurred_losses) AS total_incurred_losses,
        AVG(loss_ratio) AS avg_loss_ratio
    FROM payer_gold.loss_ratios
    GROUP BY specialty, state
    ORDER BY avg_loss_ratio DESC;
QUIT;
```

## Actuarial Example 2: Claims Trending Analysis

### 🎯 Business Question
**"What are our monthly claim trends? Are claims trending up or down?"**

This is crucial for:
- **Rate making** (applying trend factors)
- **Reserving** (projecting ultimate losses)
- **Budgeting** (forecasting next year's costs)

### 📝 What We're Calculating

```
Month-over-Month Growth = (This Month - Last Month) / Last Month
Year-over-Year Growth = (This Month - Same Month Last Year) / Same Month Last Year
```

### 🔧 SAS Equivalent
In SAS, you might use **PROC EXPAND** or **LAG functions** in a DATA step. 

In Databricks, we use **Window Functions** - specifically `LAG()` and `LEAD()`.

Let's build it! 👇

Wait, I don't know what is  `LAG()` function - let's ask Assistant!


In [0]:
%sql
-- ACTUARIAL ANALYSIS: Monthly Claims Trending
-- Window functions for MoM and YoY calculations

CREATE OR REPLACE TABLE payer_gold.claims_trend_analysis AS
WITH monthly_claims AS (
    -- Step 1: Aggregate claims by month
    SELECT 
        DATE_TRUNC('MONTH', claim_date) AS claim_month,
        YEAR(claim_date) AS claim_year,
        MONTH(claim_date) AS claim_month_num,
        COUNT(*) AS claim_count,
        SUM(total_charge) AS total_incurred,
        ROUND(AVG(total_charge), 2) AS avg_claim_cost
    FROM payer_silver.claims
    GROUP BY claim_month, claim_year, claim_month_num
)
SELECT 
    claim_month,
    claim_count,
    total_incurred,
    avg_claim_cost,
    
    -- Month-over-Month Comparison
    LAG(total_incurred, 1) OVER (ORDER BY claim_month) AS prior_month_incurred,
    ROUND((total_incurred - LAG(total_incurred, 1) OVER (ORDER BY claim_month)) / 
          LAG(total_incurred, 1) OVER (ORDER BY claim_month) * 100, 2) AS mom_growth_pct,
    
    -- Year-over-Year Comparison (12 months ago)
    LAG(total_incurred, 12) OVER (ORDER BY claim_month) AS prior_year_incurred,
    ROUND((total_incurred - LAG(total_incurred, 12) OVER (ORDER BY claim_month)) / 
          LAG(total_incurred, 12) OVER (ORDER BY claim_month) * 100, 2) AS yoy_growth_pct,
    
    -- 3-Month Moving Average (for smoothing)
    ROUND(AVG(total_incurred) OVER (ORDER BY claim_month 
                                     ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) AS moving_avg_3mo
    
FROM monthly_claims
ORDER BY claim_month;

-- Display the results
SELECT * FROM payer_gold.claims_trend_analysis;


## Actuarial Example 3: Claims Development Triangle

### 🎯 Business Question
**"How do claims develop over time? What are our age-to-age factors?"**

This is **THE fundamental tool** for actuarial reserving!

### 📝 What We're Building

A **development triangle** shows:
- **Rows**: Accident/Policy Year
- **Columns**: Development Period (months since occurrence)
- **Values**: Cumulative claims at each development point

Then we calculate:
- **Age-to-Age Factors** (e.g., 12-to-24 month factor)
- **Ultimate Loss Projections**

### 🔧 Why This Matters
- **IBNR Reserves**: Estimate unreported claims
- **Case Reserve Adequacy**: Check if reserves are sufficient
- **Rate Adequacy**: Are our prices sufficient?

Let's build a simple development view! 👇


In [0]:
%sql
-- ACTUARIAL ANALYSIS: Claims Development Pattern
-- Shows how claims emerge over time (by months since occurrence)

CREATE OR REPLACE TABLE payer_gold.claims_development AS
WITH claim_development AS (
    SELECT 
        c.claim_id,
        DATE_TRUNC('YEAR', c.claim_date) AS accident_year,
        c.claim_date,
        c.total_charge,
        m.effective_date AS member_effective_date,
        
        -- Development period in months (months between member effective date and claim date)
        -- This simulates months since policy inception
        DATEDIFF(MONTH, m.effective_date, c.claim_date) AS development_months,
        
        -- Group into development periods (quarterly for simplicity)
        CASE 
            WHEN DATEDIFF(MONTH, m.effective_date, c.claim_date) <= 3 THEN '0-3 months'
            WHEN DATEDIFF(MONTH, m.effective_date, c.claim_date) <= 6 THEN '4-6 months'
            WHEN DATEDIFF(MONTH, m.effective_date, c.claim_date) <= 12 THEN '7-12 months'
            WHEN DATEDIFF(MONTH, m.effective_date, c.claim_date) <= 24 THEN '13-24 months'
            ELSE '24+ months'
        END AS development_period
        
    FROM payer_silver.claims c
    INNER JOIN payer_silver.members m ON c.member_id = m.member_id
    WHERE c.claim_date >= m.effective_date  -- Claims after member enrollment
)
SELECT 
    accident_year,
    development_period,
    COUNT(*) AS claim_count,
    SUM(total_charge) AS cumulative_incurred,
    ROUND(AVG(total_charge), 2) AS avg_claim_size,
    
    -- Calculate % of claims reported in each period
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY accident_year), 2) AS pct_of_total_claims
    
FROM claim_development
GROUP BY accident_year, development_period
ORDER BY accident_year, development_period;

-- Display the triangle
SELECT * FROM payer_gold.claims_development;


## 🎯 YOUR TURN! (5 mins)
Work with Databricks Assistant and answer "How to estimate ultimate losses by accident year using the development triangle and age-to-age factors in SQL?"

## Actuarial Example 4: High-Risk Member Identification

### 🎯 Business Question
**"Which members are high-risk and driving our costs? How do we identify them for care management?"**

This analysis is critical for:
- **Care Management**: Target high-risk members for interventions
- **Reserving**: Set appropriate case reserves for known high-cost cases
- **Pricing**: Understand risk distribution for rate development
- **Provider Contracting**: Identify members who may need specialized care

### 📝 What We're Calculating

We'll use **percentile analysis** to identify members whose costs exceed the 95th percentile - a common threshold for "high-risk" classification.

**Key Metrics:**
- 95th percentile of claim costs
- Member-level total incurred
- Claim frequency by member
- Average claim severity

### 🔧 SAS Equivalent
In SAS, you might use **PROC UNIVARIATE** for percentiles and then filter:
```sas
PROC UNIVARIATE DATA=claims;
    VAR total_charge;
    OUTPUT OUT=pctiles PCTLPTS=95 PCTLPRE=P;
RUN;
```

In Databricks, we use **PERCENTILE_CONT** function combined with CTEs.

Let's build it! 👇



In [0]:
%sql
-- ACTUARIAL ANALYSIS: High-Risk Member Identification
-- Identify members exceeding 95th percentile for targeted interventions

CREATE OR REPLACE TABLE payer_gold.high_risk_members AS
WITH member_summary AS (
    -- Step 1: Aggregate claims by member
    SELECT 
        c.member_id,
        COUNT(c.claim_id) AS claim_count,
        SUM(c.total_charge) AS total_incurred,
        ROUND(AVG(c.total_charge), 2) AS avg_claim_cost,
        MIN(c.claim_date) AS first_claim_date,
        MAX(c.claim_date) AS last_claim_date,
        COUNT(DISTINCT YEAR(c.claim_date)) AS years_with_claims
    FROM payer_silver.claims c
    GROUP BY c.member_id
),
risk_threshold AS (
    -- Step 2: Calculate 95th percentile threshold
    SELECT 
        PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY total_incurred) AS p95_threshold,
        PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY total_incurred) AS p90_threshold,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY total_incurred) AS p75_threshold
    FROM member_summary
),
risk_classification AS (
    -- Step 3: Classify members by risk tier
    SELECT 
        ms.*,
        ROUND(ms.total_incurred / ms.claim_count, 2) AS severity,
        CASE 
            WHEN ms.total_incurred >= rt.p95_threshold THEN 'Critical Risk (95th+ percentile)'
            WHEN ms.total_incurred >= rt.p90_threshold THEN 'High Risk (90th-95th percentile)'
            WHEN ms.total_incurred >= rt.p75_threshold THEN 'Moderate Risk (75th-90th percentile)'
            ELSE 'Standard Risk'
        END AS risk_tier,
        rt.p95_threshold,
        rt.p90_threshold,
        rt.p75_threshold
    FROM member_summary ms
    CROSS JOIN risk_threshold rt
)
SELECT 
    rc.*,
    m.first_name,
    m.last_name,
    m.gender,
    m.birth_date,
    YEAR(CURRENT_DATE()) - YEAR(m.birth_date) AS age,
    m.plan_id,
    -- m.state removed because it does not exist
    DATEDIFF(DAY, rc.last_claim_date, CURRENT_DATE()) AS days_since_last_claim
FROM risk_classification rc
INNER JOIN payer_silver.members m ON rc.member_id = m.member_id
WHERE rc.total_incurred >= rc.p75_threshold  -- Focus on moderate risk and above
ORDER BY rc.total_incurred DESC;

-- Display top 20 high-risk members
SELECT * FROM payer_gold.high_risk_members LIMIT 20;

## Actuarial Example 5: Frequency & Severity Analysis by Demographics

### 🎯 Business Question
**"How do claims frequency and severity vary by age and gender? What are the key rating factors?"**

This is **fundamental for pricing** and understanding your book of business:
- **Rate Development**: Age/gender are primary rating factors
- **Experience Analysis**: Compare actual vs expected by demographic cell
- **Product Design**: Understand which demographics drive costs
- **Underwriting**: Identify profitable vs unprofitable segments

### 📝 What We're Calculating

```
Frequency = Number of Claims / Number of Members (claims per member per year)
Severity = Total Incurred / Number of Claims (average cost per claim)
Pure Premium = Frequency × Severity (expected cost per member)
```

### 🔧 SAS Equivalent
In SAS, you might use:
```sas
PROC SQL;
    CREATE TABLE freq_sev AS
    SELECT 
        age_group,
        gender,
        COUNT(*) as claim_count,
        COUNT(DISTINCT member_id) as member_count,
        CALCULATED claim_count / CALCULATED member_count as frequency,
        SUM(total_charge) / CALCULATED claim_count as severity
    FROM claims
    GROUP BY age_group, gender;
QUIT;
```

In Databricks, the SQL is nearly identical but with added window functions for benchmarking!

Let's build it! 👇



In [0]:
%sql
-- 💰 ACTUARIAL ANALYSIS: Frequency & Severity by Age/Gender
-- Core rating factor analysis for pricing and reserving

CREATE OR REPLACE TABLE payer_gold.freq_sev_by_demographics AS
WITH demographic_claims AS (
    -- Step 1: Categorize members into age bands
    SELECT 
        c.claim_id,
        c.member_id,
        c.total_charge,
        c.claim_date,
        m.gender,
        m.birth_date,
        YEAR(CURRENT_DATE()) - YEAR(m.birth_date) AS age,
        CASE 
            WHEN YEAR(CURRENT_DATE()) - YEAR(m.birth_date) < 18 THEN '0-17'
            WHEN YEAR(CURRENT_DATE()) - YEAR(m.birth_date) < 25 THEN '18-24'
            WHEN YEAR(CURRENT_DATE()) - YEAR(m.birth_date) < 35 THEN '25-34'
            WHEN YEAR(CURRENT_DATE()) - YEAR(m.birth_date) < 45 THEN '35-44'
            WHEN YEAR(CURRENT_DATE()) - YEAR(m.birth_date) < 55 THEN '45-54'
            WHEN YEAR(CURRENT_DATE()) - YEAR(m.birth_date) < 65 THEN '55-64'
            ELSE '65+'
        END AS age_band,
        m.plan_id
    FROM payer_silver.claims c
    INNER JOIN payer_silver.members m ON c.member_id = m.member_id
),
freq_sev_calcs AS (
    -- Step 2: Calculate frequency and severity by demographic cell
    SELECT 
        age_band,
        gender,
        COUNT(claim_id) AS claim_count,
        COUNT(DISTINCT member_id) AS member_count,
        SUM(total_charge) AS total_incurred,
        
        -- Key Actuarial Metrics
        ROUND(COUNT(claim_id) * 1.0 / COUNT(DISTINCT member_id), 4) AS frequency,
        ROUND(SUM(total_charge) / COUNT(claim_id), 2) AS severity,
        ROUND((COUNT(claim_id) * 1.0 / COUNT(DISTINCT member_id)) * 
              (SUM(total_charge) / COUNT(claim_id)), 2) AS pure_premium,
        
        -- Percentiles for distribution analysis
        ROUND(PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY total_charge), 2) AS median_claim,
        ROUND(PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY total_charge), 2) AS p90_claim
        
    FROM demographic_claims
    GROUP BY age_band, gender
)
SELECT 
    *,
    -- Relativities compared to overall average
    ROUND(frequency / AVG(frequency) OVER (), 3) AS frequency_relativity,
    ROUND(severity / AVG(severity) OVER (), 3) AS severity_relativity,
    ROUND(pure_premium / AVG(pure_premium) OVER (), 3) AS pure_premium_relativity,
    
    -- Credibility indicator (simple)
    CASE 
        WHEN claim_count >= 1000 THEN 'Full Credibility'
        WHEN claim_count >= 500 THEN 'Partial Credibility'
        WHEN claim_count >= 100 THEN 'Limited Credibility'
        ELSE 'Low Credibility'
    END AS credibility_indicator
    
FROM freq_sev_calcs
ORDER BY age_band, gender;

-- Display results
SELECT * FROM payer_gold.freq_sev_by_demographics;


## Actuarial Example 6: Data Quality Checks for Actuarial Analysis

**Objective**: Identify data quality issues that could impact your actuarial analysis.

As actuaries, you know that **garbage in = garbage out**. Before any analysis, you must check your data quality!

### 🎯 Common Data Quality Issues in Healthcare:
1. **Completeness**: Missing critical fields (claim amounts, dates, member IDs)
2. **Accuracy**: Negative claim amounts, future dates, invalid codes
3. **Consistency**: Duplicate claims, conflicting information
4. **Timeliness**: Lag in claim reporting

### 📝 What to Check:

**Part A: Completeness Check**
```sql
-- Example: Find records with missing critical data
SELECT 
    'Missing claim_id' AS issue,
    COUNT(*) AS record_count
FROM payer_silver.claims
WHERE claim_id IS NULL

UNION ALL

SELECT 
    'Missing total_charge' AS issue,
    COUNT(*) AS record_count
FROM payer_silver.claims
WHERE total_charge IS NULL

UNION ALL

SELECT 
    'Missing member_id' AS issue,
    COUNT(*) AS record_count
FROM payer_silver.claims
WHERE member_id IS NULL;
```

**Part B: Accuracy Check**
```sql
-- Example: Find data accuracy issues
SELECT 
    'Negative charges' AS issue,
    COUNT(*) AS record_count,
    SUM(total_charge) AS total_amount
FROM payer_silver.claims
WHERE total_charge < 0

UNION ALL

SELECT 
    'Future claim dates' AS issue,
    COUNT(*) AS record_count,
    NULL AS total_amount
FROM payer_silver.claims
WHERE claim_date > CURRENT_DATE()

UNION ALL

SELECT 
    'Zero dollar claims' AS issue,
    COUNT(*) AS record_count,
    0 AS total_amount
FROM payer_silver.claims
WHERE total_charge = 0;
```

**Part C: Duplicate Check**
```sql
-- Example: Find duplicate claims (same claim_id appearing multiple times)
SELECT 
    claim_id,
    COUNT(*) AS duplicate_count,
    SUM(total_charge) AS total_duplicate_amount
FROM payer_silver.claims
GROUP BY claim_id
HAVING COUNT(*) > 1
ORDER BY duplicate_count DESC;
```

```

### 💡 Actuarial Insight:
Data quality issues can significantly impact:
- **Loss ratio calculations** (missing claims = underestimated losses)
- **Trend analysis** (inconsistent reporting lags)
- **IBNR estimates** (late-reported claims)
- **Rate filings** (regulators scrutinize data quality!)

---


## 🎯 Exercise: Data Quality Checks in Gold 

Here's a complete data quality report combining all checks:


In [0]:
%sql
-- ✅ Comprehensive Data Quality Report

-- Part A: Completeness Checks
SELECT 'COMPLETENESS' AS check_category, 'Missing claim_id' AS issue, COUNT(*) AS record_count
FROM payer_silver.claims WHERE claim_id IS NULL
UNION ALL
SELECT 'COMPLETENESS', 'Missing total_charge', COUNT(*)
FROM payer_silver.claims WHERE total_charge IS NULL
UNION ALL
SELECT 'COMPLETENESS', 'Missing member_id', COUNT(*)
FROM payer_silver.claims WHERE member_id IS NULL
UNION ALL
SELECT 'COMPLETENESS', 'Missing claim_date', COUNT(*)
FROM payer_silver.claims WHERE claim_date IS NULL

UNION ALL

-- Part B: Accuracy Checks
SELECT 'ACCURACY', 'Negative charges', COUNT(*)
FROM payer_silver.claims WHERE total_charge < 0
UNION ALL
SELECT 'ACCURACY', 'Future claim dates', COUNT(*)
FROM payer_silver.claims WHERE claim_date > CURRENT_DATE()
UNION ALL
SELECT 'ACCURACY', 'Zero dollar claims', COUNT(*)
FROM payer_silver.claims WHERE total_charge = 0

UNION ALL

-- Part C: Consistency Checks (Duplicates)
SELECT 'CONSISTENCY', 'Duplicate claim_ids', COUNT(*) - COUNT(DISTINCT claim_id)
FROM payer_silver.claims

ORDER BY check_category, issue;


In [0]:
%sql
-- ✅ Data Quality Metrics with Percentages

WITH total_records AS (
    SELECT COUNT(*) AS total_count FROM payer_silver.claims
),
quality_issues AS (
    SELECT 
        SUM(CASE WHEN claim_id IS NULL THEN 1 ELSE 0 END) AS missing_claim_id,
        SUM(CASE WHEN total_charge IS NULL THEN 1 ELSE 0 END) AS missing_amount,
        SUM(CASE WHEN total_charge < 0 THEN 1 ELSE 0 END) AS negative_amounts,
        SUM(CASE WHEN claim_date > CURRENT_DATE() THEN 1 ELSE 0 END) AS future_dates,
        SUM(CASE WHEN total_charge = 0 THEN 1 ELSE 0 END) AS zero_charges,
        SUM(CASE WHEN provider_id IS NULL THEN 1 ELSE 0 END) AS missing_provider
    FROM payer_silver.claims
)
SELECT 
    t.total_count,
    q.missing_claim_id,
    ROUND(q.missing_claim_id * 100.0 / t.total_count, 2) AS pct_missing_id,
    q.missing_amount,
    ROUND(q.missing_amount * 100.0 / t.total_count, 2) AS pct_missing_amount,
    q.negative_amounts,
    ROUND(q.negative_amounts * 100.0 / t.total_count, 2) AS pct_negative,
    q.future_dates,
    ROUND(q.future_dates * 100.0 / t.total_count, 2) AS pct_future_dates,
    q.zero_charges,
    ROUND(q.zero_charges * 100.0 / t.total_count, 2) AS pct_zero_charges,
    q.missing_provider,
    ROUND(q.missing_provider * 100.0 / t.total_count, 2) AS pct_missing_provider
FROM total_records t, quality_issues q;


## Actuarial Example 7: Bias Detection in Healthcare Data

**Objective**: Detect potential biases in your data that could lead to unfair pricing or discriminatory practices.

As actuaries, you have an **ethical and regulatory obligation** to ensure your analyses are fair and unbiased. This is especially critical in healthcare!

### 🎯 Types of Bias to Watch For:

1. **Selection Bias**: Are certain populations underrepresented?
2. **Geographic Bias**: Do certain regions have systematically different patterns?
3. **Temporal Bias**: Has data collection changed over time?
4. **Provider Bias**: Do certain providers have unusual claim patterns?
5. **Demographic Bias**: Are outcomes correlated with protected characteristics?

---

### 📊 Exercise 7A: Demographic Representation Bias

**Check**: Are all demographic groups adequately represented?

```sql
-- Analyze member distribution by demographics
SELECT 
    gender,
    CASE 
        WHEN YEAR(CURRENT_DATE()) - YEAR(birth_date) < 18 THEN 'Under 18'
        WHEN YEAR(CURRENT_DATE()) - YEAR(birth_date) < 35 THEN '18-34'
        WHEN YEAR(CURRENT_DATE()) - YEAR(birth_date) < 50 THEN '35-49'
        WHEN YEAR(CURRENT_DATE()) - YEAR(birth_date) < 65 THEN '50-64'
        ELSE '65+'
    END AS age_group,
    COUNT(*) AS member_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS pct_of_total
FROM payer_silver.members
GROUP BY gender, age_group
ORDER BY gender, age_group;
```

**What to Look For:**
- Are any groups severely underrepresented (< 2% of population)?
- Are certain age/gender combinations missing?
- Could this skew your loss ratio analysis?

---

### 📊 Exercise 7B: Geographic Bias Detection

**Check**: Are certain states or cities systematically excluded or overrepresented?

```sql
-- Example: Analyze geographic distribution
WITH state_stats AS (
    SELECT 
        p.state,
        COUNT(DISTINCT c.claim_id) AS claim_count,
        COUNT(DISTINCT c.member_id) AS member_count,
        ROUND(AVG(c.total_charge), 2) AS avg_claim_cost,
        ROUND(SUM(c.total_charge), 2) AS total_incurred
    FROM payer_silver.claims c
    INNER JOIN payer_silver.providers p ON c.provider_id = p.provider_id
    GROUP BY p.state
),
overall_avg AS (
    SELECT AVG(avg_claim_cost) AS overall_avg_cost
    FROM state_stats
)
SELECT 
    s.state,
    s.claim_count,
    s.member_count,
    s.avg_claim_cost,
    o.overall_avg_cost,
    -- Deviation from average
    ROUND((s.avg_claim_cost - o.overall_avg_cost) / o.overall_avg_cost * 100, 2) AS pct_deviation_from_avg,
    -- Flag for significant deviation
    CASE 
        WHEN ABS((s.avg_claim_cost - o.overall_avg_cost) / o.overall_avg_cost) > 0.25 
        THEN '⚠️ HIGH DEVIATION'
        ELSE '✅ Normal'
    END AS bias_flag
FROM state_stats s, overall_avg o
ORDER BY ABS(s.avg_claim_cost - o.overall_avg_cost) DESC;
```

**Actuarial Questions:**
- Are high-cost states being excluded from your analysis?
- Could this bias your rate setting?
- Should you stratify by geography?

---

### 📊 Exercise 7C: Temporal Bias (Claims Reporting Lag)

**Check**: Has claim reporting behavior changed over time?

```sql
-- Detect if recent months have unusual patterns (incomplete data?)
WITH monthly_metrics AS (
    SELECT 
        DATE_TRUNC('MONTH', claim_date) AS claim_month,
        COUNT(*) AS claim_count,
        ROUND(AVG(total_charge), 2) AS avg_claim_size,
        COUNT(DISTINCT member_id) AS unique_members
    FROM payer_silver.claims
    GROUP BY claim_month
),
stats AS (
    SELECT 
        AVG(claim_count) AS avg_monthly_claims,
        STDDEV(claim_count) AS stddev_claims
    FROM monthly_metrics
    WHERE claim_month < DATE_TRUNC('MONTH', ADD_MONTHS(CURRENT_DATE(), -1))  -- Exclude current month
)
SELECT 
    m.claim_month,
    m.claim_count,
    m.avg_claim_size,
    s.avg_monthly_claims,
    -- Flag months with unusually low counts (potential incomplete data)
    CASE 
        WHEN m.claim_count < (s.avg_monthly_claims - 2 * s.stddev_claims) 
        THEN '⚠️ UNUSUALLY LOW - POTENTIAL BIAS'
        WHEN m.claim_count > (s.avg_monthly_claims + 2 * s.stddev_claims)
        THEN '⚠️ UNUSUALLY HIGH - INVESTIGATE'
        ELSE '✅ Normal'
    END AS completeness_flag
FROM monthly_metrics m, stats s
ORDER BY m.claim_month DESC
LIMIT 12;  -- Last 12 months
```

**Why This Matters:**
- Recent months may have incomplete data (IBNR!)
- Including incomplete months will **understate** your loss ratios
- Critical for reserving and trend analysis

---

### 📊 Exercise 7D: Provider Bias Detection

**Check**: Are certain providers outliers? Could this indicate fraud, coding issues, or data errors?

```sql
-- Example: Identify provider outliers
WITH provider_metrics AS (
    SELECT 
        p.provider_id,
        p.provider_name,
        p.specialty,
        COUNT(c.claim_id) AS claim_count,
        ROUND(AVG(c.total_charge), 2) AS avg_claim_cost,
        ROUND(SUM(c.total_charge), 2) AS total_billed
    FROM payer_silver.claims c
    INNER JOIN payer_silver.providers p ON c.provider_id = p.provider_id
    GROUP BY p.provider_id, p.provider_name, p.specialty
    HAVING COUNT(c.claim_id) >= 10  -- Minimum credibility
),
specialty_benchmarks AS (
    SELECT 
        specialty,
        AVG(avg_claim_cost) AS specialty_avg,
        STDDEV(avg_claim_cost) AS specialty_stddev
    FROM provider_metrics
    GROUP BY specialty
)
SELECT 
    pm.provider_id,
    pm.provider_name,
    pm.specialty,
    pm.claim_count,
    pm.avg_claim_cost,
    sb.specialty_avg,
    -- Calculate Z-score (standard deviations from mean)
    ROUND((pm.avg_claim_cost - sb.specialty_avg) / NULLIF(sb.specialty_stddev, 0), 2) AS z_score,
    -- Flag outliers
    CASE 
        WHEN (pm.avg_claim_cost - sb.specialty_avg) / NULLIF(sb.specialty_stddev, 0) > 3 
        THEN '🚨 EXTREME HIGH - INVESTIGATE'
        WHEN (pm.avg_claim_cost - sb.specialty_avg) / NULLIF(sb.specialty_stddev, 0) > 2 
        THEN '⚠️ HIGH OUTLIER'
        WHEN (pm.avg_claim_cost - sb.specialty_avg) / NULLIF(sb.specialty_stddev, 0) < -2 
        THEN '⚠️ LOW OUTLIER'
        ELSE '✅ Normal'
    END AS outlier_flag
FROM provider_metrics pm
INNER JOIN specialty_benchmarks sb ON pm.specialty = sb.specialty
WHERE ABS((pm.avg_claim_cost - sb.specialty_avg) / NULLIF(sb.specialty_stddev, 0)) > 2  -- Only show outliers
ORDER BY ABS((pm.avg_claim_cost - sb.specialty_avg) / NULLIF(sb.specialty_stddev, 0)) DESC;
```

**Actuarial Actions:**
- Investigate extreme outliers (fraud? coding errors?)
- Consider excluding outliers from benchmarks
- Document your methodology for rate filings

---

### 💡 Actuarial Ethics & Compliance:

**Why This Matters:**
1. **Regulatory Compliance**: ACA prohibits discrimination based on protected characteristics
2. **Actuarial Standards of Practice (ASOP)**: Require documentation of data quality and potential biases
3. **Rate Filing Requirements**: Regulators will question biased or incomplete data
4. **Professional Ethics**: Actuaries must ensure fairness in pricing and risk selection

**Best Practices:**
- ✅ Document all data limitations and potential biases
- ✅ Stratify analysis by key demographics to detect disparities
- ✅ Use credibility weighting for small segments
- ✅ Clearly communicate assumptions and limitations to stakeholders

---


# AI/BI

Intelligent analytics for everyone!

Databricks AI/BI is a new type of business intelligence product designed to provide a deep understanding of your data's semantics, enabling self-service data analysis for everyone in your organization. AI/BI is built on a compound AI system that draws insights from the full lifecycle of your data across the Databricks platform, including ETL pipelines, lineage, and other queries.

<img src="https://www.databricks.com/sites/default/files/2025-05/hero-image-ai-bi-v2-2x.png?v=1748417271" alt="Managed Tables" width="600" height="500">

# Genie

Talk with your data

Now everyone can get insights from data simply by asking questions in natural language.

<img src="https://www.databricks.com/sites/default/files/2025-06/ai-bi-genie-hero.png?v=1749162682" alt="Managed Tables" width="600" height="500">


# 🎓 Workshop Summary - You Did It!

## 🎉 Congratulations, Actuaries!

You've just completed your first Databricks workshop! Let's review what you learned.

---

## ✅ What You Accomplished Today

### 1. **Transitioned from SAS to Databricks** 🔄
- Learned how your SAS knowledge transfers
- Ran SQL queries (very similar to PROC SQL!)
- Used window functions (LAG, LEAD for trending)

### 2. **Built Actuarial Analytics** 📊
- ✅ **Loss Ratios** by segment
- ✅ **Claims Trending** (month-over-month growth)
- ✅ **Development Patterns** (claims emergence)
- ✅ **Risk Segmentation** (high-cost members)
- ✅ **Provider Analysis** (cost drivers)

### 3. **Learned Key SQL Techniques** 💻
- `GROUP BY` for aggregations (like PROC MEANS)
- `JOIN` for combining tables
- `CASE WHEN` for conditional logic (like IF-THEN)
- `LAG/LEAD` for trending (like SAS LAG functions)
- `PERCENTILE_CONT` for quantiles (like PROC UNIVARIATE)
- `WINDOW FUNCTIONS` for running calculations

---

## 🚀 How to Use This at Work

### Immediate Applications:

1. **Quarterly Loss Ratio Reports**
   - Use the loss ratio queries we built
   - Group by state, specialty, plan type
   - Export to dashboards (no more Excel!)

2. **Monthly Trending Analysis**
   - Monitor claims frequency and severity trends
   - Identify unusual spikes early
   - Feed into your pricing models

3. **Reserving Support**
   - Build development triangles
   - Calculate age-to-age factors
   - Track IBNR emergence patterns

4. **Risk Management**
   - Identify high-risk members
   - Segment populations for care management
   - Calculate risk scores

5. **Rate Filings**
   - Trend historical claims
   - Support rate change justifications
   - Build exhibits for regulators

---

## 💡 Key Takeaways

### 1. **You Already Know More Than You Think!**
If you know SAS PROC SQL, you know 90% of Databricks SQL. The syntax is almost identical!

### 2. **Start Simple**
Don't try to learn everything at once. Start with:
- Basic `SELECT` queries
- Simple aggregations (`GROUP BY`)
- Joins

Then gradually add:
- Window functions
- CTEs (WITH clauses)
- Advanced analytics

### 3. **SQL is Enough for Most Actuarial Work**
You don't need to learn Python/PySpark right away. Most actuarial analyses can be done with SQL alone!

### 4. **Iterate and Improve**
Your first queries won't be perfect. That's okay! 
- Start with something that works
- Refine it over time
- Ask for feedback

### 5. **Refer to the Best Practices!**
Best practices notebook: _**[Reference] Best Practices**_

---

## 📝 Feedback

We'd love to hear your thoughts on this workshop!

**What worked well?** What could be improved? **What topics do you want to learn next?**

---

## 🙏 Thank You!

Thank you for participating in this workshop. We hope you found it valuable and are excited to continue your Databricks journey! 🚀

---

